In [12]:
import numpy as np 
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter

In [13]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [14]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [15]:
list_comp = ['input_width', 'input_size','nb_params_conv', 'nb_params_bn', 'nb_params_fc', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']

list_var = ['input_width', 'nb_params_bn', 'nb_params_fc','nb_conv_layers', 'nb_bn_layers', 'nb_fc_layers']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [16]:
y_train = train_dataset.pop('ACET')
X_train = train_dataset
X_train.tail()

,Model_name,input_shape,input_size,nb_params_conv,flops,nb_layers,sum_activations,weighted_sum_neurons
1525,basic_model_64_15,"(1, 64, 64, 3)",12288,37300460,10696805810,27,3050376,5309448448
1526,basic_model_32_44,"(1, 32, 32, 3)",3072,32517012,23311729169,74,5247440,11618736064
1527,basic_model_112_17,"(1, 112, 112, 3)",37632,15046344,51190739064,33,25374078,25571214384
1528,basic_model_75_37,"(1, 75, 75, 3)",16875,18939872,26945702640,64,9258839,13449790920
1529,basic_model_32_23,"(1, 32, 32, 3)",3072,16689184,34211079000,41,6237204,17086005248


In [17]:
Y_e1 = test_dataset1.pop('ACET')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('ACET')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('ACET')
X_e3 = test_dataset3

In [18]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [19]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [20]:
steps = [('scaler', StandardScaler()), ('SVR', SVR())]
pipeline = Pipeline(steps)

In [57]:
"""
parameteres = {'SVR__kernel':['linear'], 
'SVR__cache_size':[5000000], 
'SVR__C':[5, 6, 8, 9, 10, 15, 20, 30, 40, 50, 100, 200, 500], 
#'SVR__gamma':[.1, .3, .5, .7, .8, .9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
'SVR__epsilon':[.1, .5, 1., 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]}

"""
# Optimal hyperparametrs
parameteres = [
               {'SVR__kernel':['linear'], 'SVR__cache_size':[50000000], 'SVR__C':[20], 'SVR__epsilon':[5.9]}]


In [58]:
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=2)

In [59]:
grid.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('SVR', SVR())]),
             param_grid=[{'SVR__C': [20], 'SVR__cache_size': [50000000],
                          'SVR__epsilon': [5.9], 'SVR__kernel': ['linear']}])

In [60]:
print(grid.best_params_)

{'SVR__C': 20, 'SVR__cache_size': 50000000, 'SVR__epsilon': 5.9, 'SVR__kernel': 'linear'}


In [61]:
best_pipe = grid.best_estimator_
best_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('SVR',
                 SVR(C=20, cache_size=50000000, epsilon=5.9, kernel='linear'))])

In [62]:
y_pred1 = best_pipe.predict(X_e1.values)
y_pred2 = best_pipe.predict(X_e2.values)
y_pred3 = best_pipe.predict(X_e3.values)

In [76]:
y_test = y_test1
y_pred = y_pred1
test_models = X_test_model_names1.values
test_input_shapes = X_test_input_shapes1.values

In [77]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD :', median_absolute_error(y_test, y_pred))

Explained variance :  0.9790499425328383
R2_value :  0.9778641266145822
Mean absolute error MAE : 29.450123056442987
Root Mean squared error RMSE : 52.530693057002345
Median absolute error MAD : 13.950152378097755


In [78]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE :', max(mape))
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE : 69.56158892624202
MAPE: 14.97 %.
Accuracy: 85.03 %.


In [79]:
import csv
with open('./measured_vs_predicted_SVR_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'real_exec_time', 'predicted_exec_time', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])